In [265]:
import torch
import torch.nn as nn 
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
import torch.nn.functional as F


In [266]:
wine_data = pd.read_csv('https://raw.githubusercontent.com/aniruddhachoudhury/Red-Wine-Quality/master/winequality-red.csv')
wine_data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [267]:
#normalizing input features

wine_data_inputs = (wine_data-wine_data.min())/(wine_data.max()-wine_data.min())
wine_data_inputs

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0.247788,0.397260,0.00,0.068493,0.106845,0.140845,0.098940,0.567548,0.606299,0.137725,0.153846,0.4
1,0.283186,0.520548,0.00,0.116438,0.143573,0.338028,0.215548,0.494126,0.362205,0.209581,0.215385,0.4
2,0.283186,0.438356,0.04,0.095890,0.133556,0.197183,0.169611,0.508811,0.409449,0.191617,0.215385,0.4
3,0.584071,0.109589,0.56,0.068493,0.105175,0.225352,0.190813,0.582232,0.330709,0.149701,0.215385,0.6
4,0.247788,0.397260,0.00,0.068493,0.106845,0.140845,0.098940,0.567548,0.606299,0.137725,0.153846,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,0.141593,0.328767,0.08,0.075342,0.130217,0.436620,0.134276,0.354626,0.559055,0.149701,0.323077,0.4
1595,0.115044,0.294521,0.10,0.089041,0.083472,0.535211,0.159011,0.370778,0.614173,0.257485,0.430769,0.6
1596,0.150442,0.267123,0.13,0.095890,0.106845,0.394366,0.120141,0.416300,0.535433,0.251497,0.400000,0.6
1597,0.115044,0.359589,0.12,0.075342,0.105175,0.436620,0.134276,0.396476,0.653543,0.227545,0.276923,0.4


In [268]:
X = wine_data_inputs.drop('quality', axis = 1)
y = wine_data['quality']

In [269]:
X = X.to_numpy()
y = y.to_numpy()

In [270]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 42, shuffle = True)

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

y_train = y_train
y_test = y_test

In [271]:
#defining model

class model(nn.Module):
    def __init__(self, in_features = 11, h1_output = 32, h2_output = 16, h3_output = 16, out_features = 10):
        super().__init__()
        self.h1 = nn.Linear(in_features, h1_output)
        self.h2 = nn.Linear(h1_output, h2_output)
        self.h3 = nn.Linear(h2_output, h3_output)
        self.out = nn.Linear(h2_output, out_features)
    
    def forward(self, X):
        X = F.relu(self.h1(X))
        X = F.relu(self.h2(X))
        X = F.relu(self.h3(X))
        X = F.softmax(self.out(X), dim = 1)

        return X

In [272]:
model = model()

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = .001)

In [273]:
#training loop

epochs = 500
losses = []

for epoch in range(epochs):
    y_pred = model.forward(X_train)

    loss = loss_fn(y_pred, y_train)

    losses.append(loss.detach().numpy())

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    if(epoch%10 == 0):
        print(f'Epoch: {epoch}, loss: {loss.detach().numpy()}')


Epoch: 0, loss: 2.307750701904297
Epoch: 10, loss: 2.3033316135406494
Epoch: 20, loss: 2.3000056743621826
Epoch: 30, loss: 2.296481132507324
Epoch: 40, loss: 2.2915782928466797
Epoch: 50, loss: 2.2837207317352295
Epoch: 60, loss: 2.269759178161621
Epoch: 70, loss: 2.2431955337524414
Epoch: 80, loss: 2.1904046535491943
Epoch: 90, loss: 2.108720541000366
Epoch: 100, loss: 2.050142526626587
Epoch: 110, loss: 2.034067153930664
Epoch: 120, loss: 2.0307374000549316
Epoch: 130, loss: 2.029514789581299
Epoch: 140, loss: 2.028388500213623
Epoch: 150, loss: 2.0264976024627686
Epoch: 160, loss: 2.0223703384399414
Epoch: 170, loss: 2.013993740081787
Epoch: 180, loss: 2.007857322692871
Epoch: 190, loss: 2.0045993328094482
Epoch: 200, loss: 2.0013489723205566
Epoch: 210, loss: 1.9975658655166626
Epoch: 220, loss: 1.9929684400558472
Epoch: 230, loss: 1.987004280090332
Epoch: 240, loss: 1.9793782234191895
Epoch: 250, loss: 1.9699697494506836
Epoch: 260, loss: 1.9589974880218506
Epoch: 270, loss: 1.947

In [274]:
#testing data accuracy

with torch.no_grad():
    correct = 0
    y_test_pred = model.forward(X_test)
    for i in range(y_test_pred.shape[0]):
        if(y_test_pred[i].argmax() == y_test[i]):
            correct += 1

print(f'Accuracy: {correct}/{y_test_pred.shape[0]}')

Accuracy: 181/320


In [275]:
#training data accuracy

with torch.no_grad():
    correct = 0
    y_train_pred = model.forward(X_train)
    for i in range(y_train_pred.shape[0]):
        if(y_train_pred[i].argmax() == y_train[i]):
            correct += 1

print(f'Accuracy: {correct}/{y_train_pred.shape[0]}')

Accuracy: 766/1279
